<a href="https://colab.research.google.com/github/NicoGangi5/Building_Deep_Learning_Models_with_TensorFlow/blob/main/RecurrentNetworksAndLSTMInDeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import numpy as np
import tensorflow as tf

In [ ]:
!mkdir data
!mkdir data/ptb
!wget -q -O data/ptb/reader.py https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0120EN-SkillsNetwork/labs/Week3/data/ptb/reader.py
!cp data/ptb/reader.py . 

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/ptb’: File exists


In [ ]:
import reader

# Building the LSTM model for Language Modeling

In [ ]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz 
!tar xzf simple-examples.tgz -C data/

--2020-12-11 19:03:31--  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vutbr.cz (www.fit.vutbr.cz)... 147.229.9.23, 2001:67c:1220:809::93e5:917
Connecting to www.fit.vutbr.cz (www.fit.vutbr.cz)|147.229.9.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34869662 (33M) [application/x-gtar]
Saving to: ‘simple-examples.tgz’

simple-examples.tgz 100%[===================>]  33.25M  2.99MB/s    in 14s     

2020-12-11 19:03:46 (2.43 MB/s) - ‘simple-examples.tgz’ saved [34869662/34869662]



In [ ]:
#Initial weight scale
init_scale = 0.1
#Initial learning rate
learning_rate = 1.0
#Maximum permissible norm for the gradient (For gradient clipping -- another measure against Exploding Gradients)
max_grad_norm = 5
#The number of layers in our model
num_layers = 2
#The total number of recurrence steps, also known as the number of layers when our RNN is "unfolded"
num_steps = 20
#The number of processing units (neurons) in the hidden layers
hidden_size_l1 = 256
hidden_size_l2 = 128
#The maximum number of epochs trained with the initial learning rate
max_epoch_decay_lr = 4
#The total number of epochs in training
max_epoch = 15
#The probability for keeping data in the Dropout Layer (This is an optimization, but is outside our scope for this notebook!)
#At 1, we ignore the Dropout Layer wrapping.
keep_prob = 1
#The decay for the learning rate
decay = 0.5
#The size for each batch of data
batch_size = 30
#The size of our vocabulary
vocab_size = 10000
embeding_vector_size= 200
#Training flag to separate training from testing
is_training = 1
#Data directory for our dataset
data_dir = "data/simple-examples/data/"

# Training data

In [ ]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, vocab, word_to_id = raw_data

In [ ]:
len(train_data)

929589

In [ ]:
def id_to_word(id_list):
    line = []
    for w in id_list:
        for word, wid in word_to_id.items():
            if wid == w:
                line.append(word)
    return line            
                

print(id_to_word(train_data[0:100]))

['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', '<eos>', 'pierre', '<unk>', 'N', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'N', '<eos>', 'mr.', '<unk>', 'is', 'chairman', 'of', '<unk>', 'n.v.', 'the', 'dutch', 'publishing', 'group', '<eos>', 'rudolph', '<unk>', 'N', 'years', 'old', 'and', 'former', 'chairman', 'of', 'consolidated', 'gold', 'fields', 'plc', 'was', 'named', 'a', 'nonexecutive', 'director', 'of', 'this', 'british', 'industrial', 'conglomerate', '<eos>', 'a', 'form', 'of', 'asbestos', 'once', 'used', 'to', 'make', 'kent', 'cigarette', 'filters', 'has', 'caused', 'a', 'high', 'percentage', 'of', 'cancer', 'deaths', 'among', 'a', 'group', 'of']


In [ ]:
itera = reader.ptb_iterator(train_data, batch_size, num_steps)
first_touple = itera.__next__()
_input_data = first_touple[0]
_targets = first_touple[1]

In [ ]:
_input_data.shape

(30, 20)

In [ ]:
_targets.shape

(30, 20)

In [ ]:
_input_data[0:3]

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605],
       [   0, 1071,    4,    0,  185,   24,  368,   20,   31, 3109,  954,
          12,    3,   21,    2, 2915,    2,   12,    3,   21]],
      dtype=int32)

In [ ]:
print(id_to_word(_input_data[0,:]))

['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim']


## Embeddings

In [ ]:
embedding_layer = tf.keras.layers.Embedding(vocab_size, embeding_vector_size,batch_input_shape=(batch_size, num_steps),trainable=True,name="embedding_vocab")  

In [ ]:
# Define where to get the data for our embeddings from
inputs = embedding_layer(_input_data)
inputs

<tf.Tensor: shape=(30, 20, 200), dtype=float32, numpy=
array([[[-0.03470422,  0.04146106, -0.0297532 , ...,  0.01125786,
          0.04887327,  0.02209339],
        [ 0.0120829 , -0.00110837,  0.00761901, ..., -0.021329  ,
         -0.03507759, -0.00708295],
        [-0.00756492, -0.02396212,  0.03685098, ...,  0.02290315,
         -0.04576325,  0.04717282],
        ...,
        [-0.02922853,  0.04947071,  0.01766041, ..., -0.01381894,
         -0.00383085, -0.02138679],
        [ 0.01283851, -0.02183471, -0.01635564, ..., -0.01462068,
          0.04689045, -0.02046909],
        [-0.01669873, -0.03602555,  0.0236362 , ..., -0.04913708,
         -0.00452728, -0.04101343]],

       [[ 0.03194896, -0.02194725, -0.00658724, ..., -0.04316182,
          0.03220231,  0.00898095],
        [ 0.02773329,  0.0083137 ,  0.00162787, ..., -0.00441587,
          0.03186499, -0.00684167],
        [-0.03177078, -0.00841136, -0.00976527, ...,  0.02787569,
         -0.04511375, -0.00850808],
        ...,

In [ ]:
# Define where to get the data for our embeddings from
inputs = embedding_layer(_input_data)
inputs

<tf.Tensor: shape=(30, 20, 200), dtype=float32, numpy=
array([[[-0.03470422,  0.04146106, -0.0297532 , ...,  0.01125786,
          0.04887327,  0.02209339],
        [ 0.0120829 , -0.00110837,  0.00761901, ..., -0.021329  ,
         -0.03507759, -0.00708295],
        [-0.00756492, -0.02396212,  0.03685098, ...,  0.02290315,
         -0.04576325,  0.04717282],
        ...,
        [-0.02922853,  0.04947071,  0.01766041, ..., -0.01381894,
         -0.00383085, -0.02138679],
        [ 0.01283851, -0.02183471, -0.01635564, ..., -0.01462068,
          0.04689045, -0.02046909],
        [-0.01669873, -0.03602555,  0.0236362 , ..., -0.04913708,
         -0.00452728, -0.04101343]],

       [[ 0.03194896, -0.02194725, -0.00658724, ..., -0.04316182,
          0.03220231,  0.00898095],
        [ 0.02773329,  0.0083137 ,  0.00162787, ..., -0.00441587,
          0.03186499, -0.00684167],
        [-0.03177078, -0.00841136, -0.00976527, ...,  0.02787569,
         -0.04511375, -0.00850808],
        ...,

# Constructing Recurrent Neural Networks

In [ ]:
lstm_cell_l1 = tf.keras.layers.LSTMCell(hidden_size_l1)
lstm_cell_l2 = tf.keras.layers.LSTMCell(hidden_size_l2)

In [ ]:
stacked_lstm = tf.keras.layers.StackedRNNCells([lstm_cell_l1, lstm_cell_l2])

In [ ]:
layer  =  tf.keras.layers.RNN(stacked_lstm,[batch_size, num_steps],return_state=False,stateful=True,trainable=True)

*_initial_state*

In [ ]:
init_state = tf.Variable(tf.zeros([batch_size,embeding_vector_size]),trainable=False)

In [ ]:
layer.inital_state = init_state

In [ ]:
layer.inital_state

<tf.Variable 'Variable:0' shape=(30, 200) dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [ ]:
outputs = layer(inputs)

In [ ]:
outputs

<tf.Tensor: shape=(30, 20, 128), dtype=float32, numpy=
array([[[ 1.2238341e-03,  1.1477596e-03, -1.1290435e-03, ...,
          4.9584854e-05,  1.1587004e-03,  9.0458069e-04],
        [ 1.0300616e-03,  2.0990944e-03,  4.5994157e-06, ...,
          6.1789498e-04,  2.4480091e-03,  1.6740604e-03],
        [ 4.9890945e-04,  3.2567137e-03,  1.4662925e-04, ...,
          2.3462274e-04,  2.1329196e-03,  1.8225710e-03],
        ...,
        [-3.5142207e-03, -5.0677685e-03, -6.8256077e-03, ...,
         -6.1697378e-03,  6.6987243e-03,  4.7551180e-04],
        [-3.1958537e-03, -5.1678289e-03, -8.9228461e-03, ...,
         -7.4503357e-03,  8.8124685e-03,  4.5684428e-04],
        [-2.4932197e-03, -3.3270491e-03, -8.3657783e-03, ...,
         -7.4896696e-03,  1.0019653e-02,  7.6640816e-04]],

       [[-7.2063856e-05,  7.0347532e-04,  8.3370350e-04, ...,
          1.5397972e-03,  6.2558422e-04,  7.5539625e-05],
        [ 8.7779248e-04,  9.9938094e-05,  1.2810752e-03, ...,
          2.1919790e-03,  1.

## Dense layer

In [ ]:
dense = tf.keras.layers.Dense(vocab_size)

In [ ]:
logits_outputs  = dense(outputs)

In [ ]:
print("shape of the output from dense layer: ", logits_outputs.shape) #(batch_size, sequence_length, vocab_size)

shape of the output from dense layer:  (30, 20, 10000)


# Activation layer

In [ ]:
activation = tf.keras.layers.Activation('softmax')

In [ ]:
output_words_prob = activation(logits_outputs)

In [ ]:
print("shape of the output from the activation layer: ", output_words_prob.shape) #(batch_size, sequence_length, vocab_size)

shape of the output from the activation layer:  (30, 20, 10000)


In [ ]:
print("The probability of observing words in t=0 to t=20", output_words_prob[0,0:num_steps])

The probability of observing words in t=0 to t=20 tf.Tensor(
[[9.99981130e-05 1.00021389e-04 9.99983968e-05 ... 1.00013887e-04
  1.00011486e-04 1.00013422e-04]
 [9.99760887e-05 1.00042576e-04 1.00007572e-04 ... 1.00018158e-04
  1.00021694e-04 1.00002209e-04]
 [9.99797048e-05 1.00044599e-04 1.00010184e-04 ... 1.00031801e-04
  1.00009536e-04 9.99668337e-05]
 ...
 [9.99340482e-05 9.99844196e-05 1.00004647e-04 ... 9.99402182e-05
  9.99952681e-05 1.00037767e-04]
 [9.99275653e-05 1.00018886e-04 1.00000463e-04 ... 9.99706390e-05
  9.99708864e-05 1.00048099e-04]
 [9.99247495e-05 1.00047546e-04 1.00029123e-04 ... 9.99819240e-05
  9.99589902e-05 1.00047633e-04]], shape=(20, 10000), dtype=float32)


# Prediction

In [ ]:
np.argmax(output_words_prob[0,0:num_steps], axis=1)

array([6042,  551, 1572, 7799, 7799, 8295, 8295, 6017, 7368, 7368, 9098,
       9098, 9098, 9243, 3036, 3036, 3036, 3036, 4846, 6294])

In [ ]:
_targets[0]

array([9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986,
       9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996], dtype=int32)

*Objetive function*

In [ ]:
def crossentropy(y_true, y_pred):
    return tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

In [ ]:
loss  = crossentropy(_targets, output_words_prob)

In [ ]:
loss[0,:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([9.21044 , 9.210327, 9.209923, 9.210556, 9.21046 , 9.209729,
       9.210798, 9.210621, 9.210139, 9.210799], dtype=float32)>

In [ ]:
cost = tf.reduce_sum(loss / batch_size)
cost

<tf.Tensor: shape=(), dtype=float32, numpy=184.20639>

# Training 

*1. Define Optimizer*

In [ ]:
# Create a variable for the learning rate
lr = tf.Variable(0.0, trainable=False)
optimizer = tf.keras.optimizers.SGD(lr=lr, clipnorm=max_grad_norm)

*2. Assemble layers to build model*

In [ ]:
model = tf.keras.Sequential()
model.add(embedding_layer)
model.add(layer)
model.add(dense)
model.add(activation)
model.compile(loss=crossentropy, optimizer=optimizer)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_vocab (Embedding)  (30, 20, 200)             2000000   
_________________________________________________________________
rnn (RNN)                    (30, 20, 128)             671088    
_________________________________________________________________
dense (Dense)                (30, 20, 10000)           1290000   
_________________________________________________________________
activation (Activation)      (30, 20, 10000)           0         
Total params: 3,961,088
Trainable params: 3,955,088
Non-trainable params: 6,000
_________________________________________________________________


*3. Trainable Variables*

In [ ]:
# Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
tvars = model.trainable_variables

In [ ]:
[v.name for v in tvars] 

['embedding_vocab/embeddings:0',
 'rnn/stacked_rnn_cells/lstm_cell/kernel:0',
 'rnn/stacked_rnn_cells/lstm_cell/recurrent_kernel:0',
 'rnn/stacked_rnn_cells/lstm_cell/bias:0',
 'rnn/stacked_rnn_cells/lstm_cell_1/kernel:0',
 'rnn/stacked_rnn_cells/lstm_cell_1/recurrent_kernel:0',
 'rnn/stacked_rnn_cells/lstm_cell_1/bias:0',
 'dense/kernel:0',
 'dense/bias:0']

*4. Calculate the gradients based on the loss function*

In [ ]:
x = tf.constant(1.0)
y =  tf.constant(2.0)
with tf.GradientTape(persistent=True) as g:
    g.watch(x)
    g.watch(y)
    func_test = 2 * x * x + 3 * x * y

In [ ]:
var_grad = g.gradient(func_test, x) # Will compute to 10.0
print(var_grad)

tf.Tensor(10.0, shape=(), dtype=float32)


In [ ]:
var_grad = g.gradient(func_test, y) # Will compute to 3.0
print(var_grad)

tf.Tensor(3.0, shape=(), dtype=float32)


In [ ]:
with tf.GradientTape() as tape:
    # Forward pass.
    output_words_prob = model(_input_data)
    # Loss value for this batch.
    loss  = crossentropy(_targets, output_words_prob)
    cost = tf.reduce_sum(loss,axis=0) / batch_size

In [ ]:
# Get gradients of loss wrt the trainable variables.
grad_t_list = tape.gradient(cost, tvars)

In [ ]:
print(grad_t_list)

[<tensorflow.python.framework.indexed_slices.IndexedSlices object at 0x7fce5715e588>, <tf.Tensor: shape=(200, 1024), dtype=float32, numpy=
array([[-4.84338216e-07,  2.80956783e-07, -2.99956810e-07, ...,
         3.19761497e-08, -3.22824491e-07, -1.17459535e-08],
       [ 1.77794141e-06, -1.36280320e-07, -1.27357453e-07, ...,
         1.64011723e-07,  3.42687656e-08,  4.17386161e-07],
       [ 2.39908815e-07,  2.51930658e-07, -5.87190016e-07, ...,
        -2.07173514e-07, -3.30247644e-07,  2.03384616e-07],
       ...,
       [-2.96242860e-07, -3.67769246e-07,  1.07804271e-06, ...,
         8.28440250e-08, -1.64169990e-07, -4.95853556e-07],
       [-3.01611806e-07, -2.30405831e-07,  1.05803656e-06, ...,
        -1.04545279e-07, -1.83397418e-07, -4.90317632e-07],
       [-6.99878456e-07, -3.63556950e-07, -9.99256855e-08, ...,
        -7.14011392e-08,  2.25640051e-07,  8.57263132e-08]], dtype=float32)>, <tf.Tensor: shape=(256, 1024), dtype=float32, numpy=
array([[-3.9737643e-08,  2.9265824

In [ ]:
# Define the gradient clipping threshold
grads, _ = tf.clip_by_global_norm(grad_t_list, max_grad_norm)
grads

 <tf.Tensor: shape=(200, 1024), dtype=float32, numpy=
 array([[-4.84338216e-07,  2.80956783e-07, -2.99956810e-07, ...,
          3.19761497e-08, -3.22824491e-07, -1.17459535e-08],
        [ 1.77794141e-06, -1.36280320e-07, -1.27357453e-07, ...,
          1.64011723e-07,  3.42687656e-08,  4.17386161e-07],
        [ 2.39908815e-07,  2.51930658e-07, -5.87190016e-07, ...,
         -2.07173514e-07, -3.30247644e-07,  2.03384616e-07],
        ...,
        [-2.96242860e-07, -3.67769246e-07,  1.07804271e-06, ...,
          8.28440250e-08, -1.64169990e-07, -4.95853556e-07],
        [-3.01611806e-07, -2.30405831e-07,  1.05803656e-06, ...,
         -1.04545279e-07, -1.83397418e-07, -4.90317632e-07],
        [-6.99878456e-07, -3.63556950e-07, -9.99256855e-08, ...,
         -7.14011392e-08,  2.25640051e-07,  8.57263132e-08]], dtype=float32)>,
 <tf.Tensor: shape=(256, 1024), dtype=float32, numpy=
 array([[-3.9737643e-08,  2.9265824e-08,  5.3763327e-09, ...,
          4.8724949e-08, -4.4655465e-08, -2

*5.Apply the optimizer to the variables/gradients tuple.*

In [ ]:
# Create the training TensorFlow Operation through our optimizer
train_op = optimizer.apply_gradients(zip(grads, tvars))

# LSTM

In [ ]:
class PTBModel(object):


    def __init__(self):
        ######################################
        # Setting parameters for ease of use #
        ######################################
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.hidden_size_l1 = hidden_size_l1
        self.hidden_size_l2 = hidden_size_l2
        self.vocab_size = vocab_size
        self.embeding_vector_size = embeding_vector_size
        # Create a variable for the learning rate
        self._lr = 1.0
        
        ###############################################################################
        # Initializing the model using keras Sequential API  #
        ###############################################################################
        
        self._model = tf.keras.models.Sequential()
        
        ####################################################################
        # Creating the word embeddings layer and adding it to the sequence #
        ####################################################################
        with tf.device("/cpu:0"):
            # Create the embeddings for our input data. Size is hidden size.
            self._embedding_layer = tf.keras.layers.Embedding(self.vocab_size, self.embeding_vector_size,batch_input_shape=(self.batch_size, self.num_steps),trainable=True,name="embedding_vocab")  #[10000x200]
            self._model.add(self._embedding_layer)
            

        ##########################################################################
        # Creating the LSTM cell structure and connect it with the RNN structure #
        ##########################################################################
        # Create the LSTM Cells. 
        # This creates only the structure for the LSTM and has to be associated with a RNN unit still.
        # The argument  of LSTMCell is size of hidden layer, that is, the number of hidden units of the LSTM (inside A). 
        # LSTM cell processes one word at a time and computes probabilities of the possible continuations of the sentence.
        lstm_cell_l1 = tf.keras.layers.LSTMCell(hidden_size_l1)
        lstm_cell_l2 = tf.keras.layers.LSTMCell(hidden_size_l2)
        

        
        # By taking in the LSTM cells as parameters, the StackedRNNCells function junctions the LSTM units to the RNN units.
        # RNN cell composed sequentially of stacked simple cells.
        stacked_lstm = tf.keras.layers.StackedRNNCells([lstm_cell_l1, lstm_cell_l2])


        

        ############################################
        # Creating the input structure for our RNN #
        ############################################
        # Input structure is 20x[30x200]
        # Considering each word is represended by a 200 dimentional vector, and we have 30 batchs, we create 30 word-vectors of size [30xx2000]
        # The input structure is fed from the embeddings, which are filled in by the input data
        # Feeding a batch of b sentences to a RNN:
        # In step 1,  first word of each of the b sentences (in a batch) is input in parallel.  
        # In step 2,  second word of each of the b sentences is input in parallel. 
        # The parallelism is only for efficiency.  
        # Each sentence in a batch is handled in parallel, but the network sees one word of a sentence at a time and does the computations accordingly. 
        # All the computations involving the words of all sentences in a batch at a given time step are done in parallel. 

        ########################################################################################################
        # Instantiating our RNN model and setting stateful to True to feed forward the state to the next layer #
        ########################################################################################################
        
        self._RNNlayer  =  tf.keras.layers.RNN(stacked_lstm,[batch_size, num_steps],return_state=False,stateful=True,trainable=True)
        
        # Define the initial state, i.e., the model state for the very first data point
        # It initialize the state of the LSTM memory. The memory state of the network is initialized with a vector of zeros and gets updated after reading each word.
        self._initial_state = tf.Variable(tf.zeros([batch_size,embeding_vector_size]),trainable=False)
        self._RNNlayer.inital_state = self._initial_state
    
        ############################################
        # Adding RNN layer to keras sequential API #
        ############################################        
        self._model.add(self._RNNlayer)
        
        #self._model.add(tf.keras.layers.LSTM(hidden_size_l1,return_sequences=True,stateful=True))
        #self._model.add(tf.keras.layers.LSTM(hidden_size_l2,return_sequences=True))
        
        
        ####################################################################################################
        # Instantiating a Dense layer that connects the output to the vocab_size  and adding layer to model#
        ####################################################################################################
        self._dense = tf.keras.layers.Dense(self.vocab_size)
        self._model.add(self._dense)
 
        
        ####################################################################################################
        # Adding softmax activation layer and deriving probability to each class and adding layer to model #
        ####################################################################################################
        self._activation = tf.keras.layers.Activation('softmax')
        self._model.add(self._activation)

        ##########################################################
        # Instantiating the stochastic gradient decent optimizer #
        ########################################################## 
        self._optimizer = tf.keras.optimizers.SGD(lr=self._lr, clipnorm=max_grad_norm)
        
        
        ##############################################################################
        # Compiling and summarizing the model stacked using the keras sequential API #
        ##############################################################################
        self._model.compile(loss=self.crossentropy, optimizer=self._optimizer)
        self._model.summary()


    def crossentropy(self,y_true, y_pred):
        return tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

    def train_batch(self,_input_data,_targets):
        #################################################
        # Creating the Training Operation for our Model #
        #################################################
        # Create a variable for the learning rate
        self._lr = tf.Variable(0.0, trainable=False)
        # Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
        tvars = self._model.trainable_variables
        # Define the gradient clipping threshold
        with tf.GradientTape() as tape:
            # Forward pass.
            output_words_prob = self._model(_input_data)
            # Loss value for this batch.
            loss  = self.crossentropy(_targets, output_words_prob)
            # average across batch and reduce sum
            cost = tf.reduce_sum(loss/ self.batch_size)
        # Get gradients of loss wrt the trainable variables.
        grad_t_list = tape.gradient(cost, tvars)
        # Define the gradient clipping threshold
        grads, _ = tf.clip_by_global_norm(grad_t_list, max_grad_norm)
        # Create the training TensorFlow Operation through our optimizer
        train_op = self._optimizer.apply_gradients(zip(grads, tvars))
        return cost
        
    def test_batch(self,_input_data,_targets):
        #################################################
        # Creating the Testing Operation for our Model #
        #################################################
        output_words_prob = self._model(_input_data)
        loss  = self.crossentropy(_targets, output_words_prob)
        # average across batch and reduce sum
        cost = tf.reduce_sum(loss/ self.batch_size)

        return cost
    @classmethod
    def instance(cls) : 
        return PTBModel()

In [ ]:

########################################################################################################################
# run_one_epoch takes as parameters  the model instance, the data to be fed, training or testing mode and verbose info #
########################################################################################################################
def run_one_epoch(m, data,is_training=True,verbose=False):

    #Define the epoch size based on the length of the data, batch size and the number of steps
    epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
    start_time = time.time()
    costs = 0.
    iters = 0
    
    m._model.reset_states()
    
    #For each step and data point
    for step, (x, y) in enumerate(reader.ptb_iterator(data, m.batch_size, m.num_steps)):
        
        #Evaluate and return cost, state by running cost, final_state and the function passed as parameter
        #y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)
        if is_training : 
            loss=  m.train_batch(x, y)
        else :
            loss = m.test_batch(x, y)
                                   

        #Add returned cost to costs (which keeps track of the total costs for this epoch)
        costs += loss
        
        #Add number of steps to iteration counter
        iters += m.num_steps

        if verbose and step % (epoch_size // 10) == 10:
            print("Itr %d of %d, perplexity: %.3f speed: %.0f wps" % (step , epoch_size, np.exp(costs / iters), iters * m.batch_size / (time.time() - start_time)))
        


    # Returns the Perplexity rating for us to keep track of how the model is evolving
    return np.exp(costs / iters)



In [ ]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, _, _ = raw_data

In [ ]:
# Instantiates the PTBModel class
m=PTBModel.instance()   
K = tf.keras.backend 
for i in range(max_epoch):
    # Define the decay for this epoch
    lr_decay = decay ** max(i - max_epoch_decay_lr, 0.0)
    dcr = learning_rate * lr_decay
    m._lr = dcr
    K.set_value(m._model.optimizer.learning_rate,m._lr)
    print("Epoch %d : Learning rate: %.3f" % (i + 1, m._model.optimizer.learning_rate))
    # Run the loop for this epoch in the training mode
    train_perplexity = run_one_epoch(m, train_data,is_training=True,verbose=True)
    print("Epoch %d : Train Perplexity: %.3f" % (i + 1, train_perplexity))
        
    # Run the loop for this epoch in the validation mode
    valid_perplexity = run_one_epoch(m, valid_data,is_training=False,verbose=False)
    print("Epoch %d : Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
    
# Run the loop in the testing mode to see how effective was our training
test_perplexity = run_one_epoch(m, test_data,is_training=False,verbose=False)
print("Test Perplexity: %.3f" % test_perplexity)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_vocab (Embedding)  (30, 20, 200)             2000000   
_________________________________________________________________
rnn_1 (RNN)                  (30, 20, 128)             671088    
_________________________________________________________________
dense_1 (Dense)              (30, 20, 10000)           1290000   
_________________________________________________________________
activation_1 (Activation)    (30, 20, 10000)           0         
Total params: 3,961,088
Trainable params: 3,955,088
Non-trainable params: 6,000
_________________________________________________________________
Epoch 1 : Learning rate: 1.000
Itr 10 of 1549, perplexity: 4915.144 speed: 1057 wps
Itr 164 of 1549, perplexity: 1087.330 speed: 1095 wps
Itr 318 of 1549, perplexity: 846.870 speed: 1097 wps
Itr 472 of 1549, perplexity: 703.999 speed: 109

KeyboardInterrupt: ignored